In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import yfinance as yf
import pandas as pd
import math
from scipy.stats import norm

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
TICKER = "AAPL"
EXPIRATION_COUNT = 3  # Number of expiration dates to fetch

# Black-Scholes Greeks Calculation with T=0 Fix
def calculate_greeks(S, K, T, r, sigma, option_type="call"):
    """Compute Black-Scholes Greeks while handling T=0 cases."""
    epsilon = 1/365  # Small constant to prevent division by zero
    T = max(T, epsilon)  # Ensure T is never exactly 0

    d1 = (math.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    delta = norm.cdf(d1) if option_type == "call" else -norm.cdf(-d1)
    gamma = norm.pdf(d1) / (S * sigma * math.sqrt(T))
    theta = (- (S * norm.pdf(d1) * sigma) / (2 * math.sqrt(T)) -
             r * K * math.exp(-r * T) * norm.cdf(d2 if option_type == "call" else -d2))
    vega = S * norm.pdf(d1) * math.sqrt(T)
    rho = K * T * math.exp(-r * T) * norm.cdf(d2 if option_type == "call" else -d2)

    return delta, gamma, theta, vega, rho


# ⚡ Fetch Live Stock Price
def fetch_stock_data(ticker):
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1d", interval="1m")
    if hist.empty:
        raise ValueError(f"No stock data found for {ticker}")
    
    latest_price = hist["Close"].iloc[-1]
    return torch.tensor(latest_price, dtype=torch.float32)

# ⚡ Fetch Option Data for Multiple Expirations
def fetch_option_data(ticker, num_expirations=EXPIRATION_COUNT):
    stock = yf.Ticker(ticker)
    expiration_dates = stock.options[:num_expirations]  # Select first `num_expirations` dates
    if not expiration_dates:
        raise ValueError(f"No options data found for {ticker}")
    
    all_options = []
    stock_price = fetch_stock_data(ticker).item()
    risk_free_rate = 0.05  # Assume 5% risk-free rate

    for expiry in expiration_dates:
        opt_chain = stock.option_chain(expiry)
        T = (pd.to_datetime(expiry) - pd.Timestamp.today()).days / 365  # Time to expiration in years
        
        for option_df, option_type in [(opt_chain.calls, "call"), (opt_chain.puts, "put")]:
            option_df["option_type"] = option_type
            option_df = option_df[["strike", "impliedVolatility", "option_type"]].dropna()
            option_df["impliedVolatility"] /= 100  # Convert to decimal

            # Compute Greeks
            greeks = option_df.apply(
                lambda row: calculate_greeks(
                    S=stock_price,
                    K=row["strike"],
                    T=T,
                    r=risk_free_rate,
                    sigma=row["impliedVolatility"],
                    option_type=row["option_type"]
                ), axis=1
            )

            option_df[["delta", "gamma", "theta", "vega", "rho"]] = pd.DataFrame(greeks.tolist(), index=option_df.index)
            option_df["expiration_days"] = T * 365  # Store expiration in days

            all_options.append(option_df)

    options = pd.concat(all_options)

    # Convert to PyTorch tensor
    option_tensor = torch.tensor(
        options[["strike", "impliedVolatility", "delta", "gamma", "theta", "vega", "rho", "expiration_days"]].values,
        dtype=torch.float32
    )

    return option_tensor

# ⚡ Combine Stock & Multi-Expiration Option Data for GAN Training
def create_market_data(ticker):
    stock_price = fetch_stock_data(ticker)
    options = fetch_option_data(ticker)
    
    # Add stock price column
    stock_prices = stock_price.expand(options.shape[0], 1)

    # Concatenate stock price with options
    market_data = torch.cat([stock_prices, options], dim=1)

    # Save as a PyTorch file
    torch.save(market_data, "market_data.pt")
    print(f"✅ Market data saved as market_data.pt | Shape: {market_data.shape}")

    return market_data

# ⚡ Fetch & Save Data
market_data = create_market_data(TICKER)
print(market_data.shape)  # Example: (300, 9) where 300 options and 9 features (stock + option data)



✅ Market data saved as market_data.pt | Shape: torch.Size([305, 9])
torch.Size([305, 9])


In [3]:
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim=4):  # 4 outputs for multi-leg options
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim),
            nn.Tanh()  # Outputs between -1 and 1 (scaled trading positions)
        )
    
    def forward(self, x):
        trades = self.model(x)
        return self.apply_strategy_constraints(trades, x)
    
    def apply_strategy_constraints(self, trades, market_state):
        """
        Apply constraints to ensure valid multi-leg strategies (Iron Condors, Straddles, Spreads)
        market_state contains: stock price, IV, portfolio Greeks, etc.
        """
        stock_price = market_state[:, 0]  # Assuming first feature is stock price
        
        # Generate valid strikes based on stock price
        atm_strike = stock_price.round()  # Closest ATM strike
        strikes = torch.stack([
        atm_strike - 5, 
        atm_strike, 
        atm_strike + 5, 
        atm_strike + 10
        ], dim=1).to(trades.device)

        # Ensure strikes follow valid order for an iron condor
        trades[:, 0] = -torch.abs(trades[:, 0])  # Sell lower call
        trades[:, 1] = torch.abs(trades[:, 1])   # Buy higher call
        trades[:, 2] = -torch.abs(trades[:, 2])  # Sell lower put
        trades[:, 3] = torch.abs(trades[:, 3])   # Buy higher put
        
        return trades


In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),  # Output a single scalar for each strategy
            nn.Sigmoid()  # Probability of being a valid trade (0 to 1)
        )
    
    def forward(self, x):
        return self.model(x)

    def evaluate_trade(self, trades, market_state):
        """
        Custom function to check if generated strategies are valid & risk-aware
        """
        # Check if iron condor or straddle is correctly structured
        strategy_validity = ((trades[:, 0] < trades[:, 1]) & (trades[:, 2] < trades[:, 3])).float()
        ##################FLAG
        # Penalize excessive delta exposure (violates delta-neutrality)
        delta_violation = torch.abs(market_state[:, -4])  # Portfolio delta
        risk_penalty = torch.clamp(delta_violation - 0.05, min=0)  # Excess delta exposure

        # Final validity score
        validity = self.forward(trades).squeeze(1)  # Flatten to a 1D tensor of shape [batch_size]
        return validity - risk_penalty


In [5]:
# Define model input dimensions
input_dim = 10  # Features: stock prices, IV, portfolio Greeks, etc.
output_dim = 4  # Number of strategy legs (for multi-leg options)

# Initialize models
G = Generator(input_dim, output_dim).to(device)  # Generator
D = Discriminator(output_dim).to(device)  # Discriminator

# Optimizers
g_optimizer = optim.Adam(G.parameters(), lr=0.001)
d_optimizer = optim.Adam(D.parameters(), lr=0.001)


In [6]:
# Define training parameters
num_epochs = 1000  
batch_size = 32  # Number of trades per batch
# Define loss function
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for GAN training

In [11]:
def train_gan(G, D, data, input_dim, output_dim, epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    G.to(device)
    D.to(device)

    optimizer_G = optim.Adam(G.parameters(), lr=0.001)
    optimizer_D = optim.Adam(D.parameters(), lr=0.001)
    criterion = nn.BCELoss()
    
    greek_constraints = {
        "delta": (-0.5, 0.5),
        "gamma": (0.0, 0.1),
        "theta": (-0.05, 0.05),
        "vega": (0.0, 0.2),
        "rho": (-0.1, 0.1)
    }
    
    for epoch in range(epochs):
        real_data = data[:32].to(device)  # Directly use the tensor without .values
        fake_trades = G(torch.randn(32, input_dim).to(device))
        
        valid_fake_trades = []
        for trade in fake_trades:
            if D.evaluate_trade(trade.cpu().detach().numpy(), greek_constraints):
                valid_fake_trades.append(trade)
        
        if valid_fake_trades:
            fake_trades = torch.stack(valid_fake_trades).to(device)
        else:
            continue
        
        real_labels = torch.ones(fake_trades.size(0), 1).to(device)
        fake_labels = torch.zeros(fake_trades.size(0), 1).to(device)
        
        D_real = D(real_data)
        D_fake = D(fake_trades.detach())
        
        loss_D = criterion(D_real, real_labels) + criterion(D_fake, fake_labels)
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()
        
        D_fake_new = D(fake_trades)
        loss_G = criterion(D_fake_new, real_labels)
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()
    
    print("Training completed.")

ticker = "AAPL"
data = fetch_option_data(ticker)
G = Generator(input_dim=8, output_dim=4)
D = Discriminator(input_dim=8)
train_gan(G, D, data, input_dim=8, output_dim=4)


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed